In [1]:
# Import libraries
import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests
from splinter import Browser
from requests import Session
from bs4 import BeautifulSoup as bs
import re
import json
from pprint import pprint

from datetime import datetime, date, time, timedelta
from time import sleep

import config

from pyvirtualdisplay import Display

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver

import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

In [2]:
# Fetch the service account key JSON file contents
cred = credentials.Certificate(config.firebase_file_path)
# Initialize the app with a service account, granting admin privileges
firebase_admin.initialize_app(cred, {
    'databaseURL': config.databaseURL
})


In [3]:
# Start Browser
driver = webdriver.Chrome("/usr/local/bin/chromedriver")
sleep(5)

# Navigate to login page
driver.get("https://www.etsy.com/signin")
sleep(5)

try:
    # Populate sign in fields
    emailElem = driver.find_element_by_id("join_neu_email_field")
    emailElem.send_keys(config.email_address)

    sleep(5)

    passwordElem = driver.find_element_by_id("join_neu_password_field")
    passwordElem.send_keys(config.etsy_password)

    sleep(5)

    # Submit sign in information
    passwordElem.submit()


    print("Successful 1st Attempt Login")
    sleep(5)


# If first login attempt is unsuccessful, refresh the page and make 2nd attempt.
except:
    # Navigate to login page
    driver.get("https://www.etsy.com/signin")
    sleep(5)

    # Populate sign in fields
    emailElem = driver.find_element_by_id("join_neu_email_field")
    emailElem.send_keys(config.email_address)
    sleep(5)

    passwordElem = driver.find_element_by_id("join_neu_password_field")
    passwordElem.send_keys(config.etsy_password)
    sleep(5)

    # Submit sign in information
    passwordElem.submit()
    sleep(5)

  
    print("Successful 2nd Attempt Login")


Successful 1st Attempt Login


In [4]:
yesterday = date.today() - timedelta(days=1)
yesterday_year = yesterday.strftime("%Y")

# ----------------- CSV DOWNLOADS ------------------#
try:
    # # Scrape - Download CSV Files
    # Navigate to download page
    driver.get("https://www.etsy.com/your/shops/jetchcreations/download")
    # Select options from dropdown lists & submit to begin download
    # After download the page will refresh to prevent system notifications
    
    select = Select(driver.find_element_by_id("filter-csv-type"))
    select.select_by_value("order-level")

    select = Select(driver.find_element_by_id("filter-year"))
    select.select_by_value(yesterday_year)

    button = driver.find_element_by_id("order-csv-form")
    button.submit()
except:
    print("error downloading csv")

In [5]:
sleep(5)
try:
    # Read in CSVs to dataframes
    sold_orders_file_path = (
        f"/Users/jessicaetchechury/Downloads/EtsySoldOrders{yesterday_year}.csv"
    )
    sold_orders_df = pd.read_csv(sold_orders_file_path)
    # Convert data types to string
    sold_orders_df = sold_orders_df.applymap(str)
    
except:
    print("error reading csv")

In [6]:
sold_orders_df2 = sold_orders_df.rename(
        columns={
            "Sale Date": "orderDate",
            "Order ID": "orderID",
            "Buyer User ID": "buyerID",
            "First Name": "firstName",
            "Last Name": "lastName",
            "Date Shipped": "dateShipped",
            "Street 1": "street1",
            "Street 2": "street2",
            "Ship City": "city",
            "Ship State": "state",
            "Ship Zipcode": "zipCode",
            "Ship Country": "country",
            "Order Value": "orderValue",
            "Coupon Code": "couponCode",
            "Coupon Details": "details",
            "Discount Amount": "discountAmount",
            "Shipping Discount": "shippingDiscount",
            "Shipping": "shipping",
            "Sales Tax": "salesTax",
            "Order Total": "orderTotal",
            "Card Processing Fees": "fees",
            "Order Net": "orderNet",
            "Adjusted Order Total": "adjustedTotal",
            "Adjusted Card Processing Fees": "adjustedFees",
            "Adjusted Net Order Amount": "adjustedNet",
            "Order Type": "orderType",
            "Payment Type": "paymentType",
        }
    )

In [7]:
orders_shipped_df = sold_orders_df2[['orderID','dateShipped']]
#delete rows that contain NaN/null - only want to import complete records
orders_shipped_df2 = orders_shipped_df[~orders_shipped_df.dateShipped.str.contains("nan")]
orders_shipped_df3 = orders_shipped_df2.applymap(str)
orders_shipped_df3.fillna("", inplace=True)

In [8]:
# Check that records for the day have not already been added
# Do the following if the dataframe is not empty
if orders_shipped_df3.empty == False:
    fb_db = "order_shipped.json"
    r = requests.get(config.databaseURL + fb_db)
    r = r.json()

    if r:
        data = [r[i] for i in r]
        df_fb = pd.DataFrame.from_dict(data, orient='columns')
        df_fb = df_fb.applymap(str)
    else:
#         data_collection_log= open(f"/Users/jessicaetchechury/Desktop/JEtchCreationsDashboard/DataCollection/logs/{yesterday_full_date}_data_collection_log.txt","a")
#         data_collection_log.write(f"\n{datetime.now()} ***ERROR- Acess Firebase order_shipped")
        print("***ERROR- Acess Firebase order_shipped")
        
    df_tups = [(item.orderID, item.dateShipped) for index, item in df_fb.iterrows()]
    
    
    ref = db.reference('order_shipped')
    # Compare tuples from Firebase database and tuples from local df.  Only add tuples that do not already exist in Firebase database
    
    # New record count
    count=0
    
    for index, item in orders_shipped_df3.iterrows():
        
        # Create tuples from local df        
        var_o=item.orderID, item.dateShipped
        
        if var_o not in df_tups:
            # Push item to db
            ref.push(item.to_dict()) 

            # Add to new record count
            count += 1

            items=item.values.tolist()

            new_val= ", ".join( repr(e) for e in items ) 

            new_val2=new_val.replace("'","")

#             txt_file = open("/Users/jessicaetchechury/Desktop/JEtchCreationsDashboard/Data/order_shipped.txt","a")
#             txt_file.write(f"\n{new_val2}") 
#             txt_file.close()            
            
        else:
            pass        

In [9]:
yesterday_full_date = yesterday.strftime("%Y-%m-%d")

# # Order_shipped DF

orders_shipped_df = sold_orders_df2[["orderID", "dateShipped"]]
# delete rows that contain NaN/null - only want to import complete records
orders_shipped_df2 = orders_shipped_df[orders_shipped_df.dateShipped.str.contains("nan")]
orders_shipped_df3 = orders_shipped_df2.applymap(str)
orders_shipped_df3.fillna("", inplace=True)

print("DF Complete- orders_shipped_df")



# Check that records for the day have not already been added
# Do the following if the dataframe is not empty
if orders_shipped_df3.empty == False:
    fb_db = "order_shipped.json"
    r = requests.get(config.databaseURL + fb_db)
    r = r.json()

    if r:
        data = [r[i] for i in r]
        df_fb = pd.DataFrame.from_dict(data, orient="columns")
        df_fb = df_fb.applymap(str)
    else:
        print("***ERROR- Acess Firebase order_shipped")

    df_tups = [(item.orderID, item.dateShipped) for index, item in df_fb.iterrows()]

    ref = db.reference("order_shipped")
    # Compare tuples from Firebase database and tuples from local df.  Only add tuples that do not already exist in Firebase database

    # New record count
    count = 0

    for index, item in orders_shipped_df3.iterrows():

        # Create tuples from local df
        var_o = item.orderID, item.dateShipped

        if var_o not in df_tups:
            # Push item to db
            ref.push(item.to_dict())

            # Add to new record count
            count += 1

            items = item.values.tolist()

            new_val = ", ".join(repr(e) for e in items)

            new_val2 = new_val.replace("'", "")

            txt_file = open(
                "/Users/jessicaetchechury/Desktop/JEtchCreationsDashboard/Data/order_shipped.txt",
                "a",
            )
            txt_file.write(f"\n{new_val2}")
            txt_file.close()

        else:
            pass

    print(f"Firebase- {count} records added to order_shipped")


else:

    print("DF EMPTY- orders_shipped_df3")

DF Complete- orders_shipped_df
Firebase- 0 records added to order_shipped


In [10]:
# ----------------- OPEN ORDERS ------------------#

# # Scrape - Open Orders
# Create lists to hold query information
open_order_id = []
open_order_query_urls = []
ship_by_date = []
order_time = []

email = []
orderID = []
buyer = []

# Navigate to page
open_order_page_url = "https://www.etsy.com/your/orders/sold?ref=seller-platform-mcnav"

driver.get(open_order_page_url)
sleep(2)

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Gather buyer contact information
try:
    for each in soup.find_all("div", class_="panel-body-row"):
        for item in each:
            split_details = list(item.stripped_strings)
            if "View user profile" in split_details:
                # Find index of generic item in list
                target_index = split_details.index("#")
                # Create new shorter list and then extract information needed
                newList = split_details[: target_index + 2]
                orderNo = newList[-1]
                orderID.append(orderNo)
                buyerName = newList[0]
                buyer.append(buyerName)
                email_address = newList[-3]
                email.append(email_address)

    buyer_contact = pd.DataFrame({"orderID": orderID, "Name": buyer, "email": email})

    # Convert all objects in dataframe to strings and fill N/A with whitespace
    buyer_contact = buyer_contact.applymap(str)
    buyer_contact.fillna("", inplace=True)

    print("DF Complete- buyer_contact")

    # Locate html to scrape
    open_orders_page = soup.find_all("div", class_="col-xs-12 col-md-8")

    for orders in open_orders_page:
        customer_name = orders.find("span", id="unsanitize")

    open_orders_page = soup.find_all(
        "h2", class_="col-xs-5 col-md-12 text-body-smaller text-gray"
    )

    # Loop through html to find order IDs to create URLS
    for orders in open_orders_page:
        open_order_url = orders.find("a", class_="text-gray")["href"]

        open_order_id_value = open_order_url.replace(
            "/your/orders/sold?ref=seller-platform-mcnav&order_id=", ""
        ).replace("/your/orders/sold?order_id=", "")
        open_order_id.append(open_order_id_value)

        open_order_url = "https://www.etsy.com" + open_order_url
        open_order_query_urls.append(open_order_url)

    # Loop through open order pages and append target info to lists
    for url in open_order_query_urls:

        driver.get(url)
        sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        tomorrow = date.today() + timedelta(days=1)

        ship_by_date_values = (
            soup.find(
                "div",
                class_="flag-img flag-img-right no-wrap text-right vertical-align-bottom hide-xs hide-sm",
            )
            .find("div", class_="strong")
            .text
        )

        ship_by_date_values = ship_by_date_values.replace("Ship by ", "")

        def validate(date_text):
            try:
                datetime.strptime(ship_by_date_values, "%b %d, %Y")
                return True
            except ValueError:
                return False

        if validate(ship_by_date_values) == True:
            d = datetime.strptime(ship_by_date_values, "%b %d, %Y")
        elif ship_by_date_values == "today":
            d = today
        else:
            d = tomorrow

        ship_by_date_values = d.strftime("%Y-%m-%d")

        ship_by_date.append(ship_by_date_values)

        order_date_values = (
            soup.find(
                "div",
                class_="flag-img flag-img-right no-wrap text-right vertical-align-bottom hide-xs hide-sm",
            )
            .find("div", class_="text-body-smaller mt-xs-1")
            .text
        )

        order_date_values = order_date_values.replace("Ordered ", "")
        order_date_values = order_date_values.strip(" ")
        d = datetime.strptime(order_date_values, "%I:%M%p, %a, %b %d, %Y")
        order_date_values = d.strftime("%m%d%y")
        order_time_values = d.strftime("%H:%M")
        order_time.append(order_time_values)


    print("Open orders scraped")

    orders_df_temp2 = pd.DataFrame(
        {
            "orderID": open_order_id,
            "orderURL": open_order_query_urls,
            "shipByDate": ship_by_date,
            "orderTime": order_time,
        }
    )
    orders_df_temp2["record_date"] = yesterday

    # Turn all objects in dataframe to strings and replace NaN with whitespace
    orders_df_temp2 = orders_df_temp2.applymap(str)
    orders_df_temp2.fillna("", inplace=True)

    print("DF Complete- orders_df_temp2")

    open_orders_df = pd.merge(orders_df_temp2, buyer_contact, on="orderID", how="inner")

    print("DF Complete- open_orders_df")

except:

    print("***ERROR: OPEN ORDERS SCRAPE")



DF Complete- buyer_contact
Open orders scraped
DF Complete- orders_df_temp2
DF Complete- open_orders_df


In [11]:
## Orders DF

# Create data frame from CSV
orders_df_temp1 = sold_orders_df2[["orderID", "orderDate", "buyerID"]]

# Only return open orders
orders_df = pd.merge(orders_df_temp1, orders_df_temp2, on="orderID", how="outer")

orders_df = orders_df[
    ["orderID", "orderDate", "orderTime", "shipByDate", "buyerID", "orderURL"]
]

orders_df["orderDate"] = pd.to_datetime(
    orders_df["orderDate"], format="%m/%d/%y", errors="coerce"
).dt.strftime("%Y-%m-%d")

orders_df = orders_df.applymap(str)
orders_df.fillna("", inplace=True)

print("DF Complete- orders_df")

DF Complete- orders_df


In [12]:
# Check that records for the day have not already been added
# Do the following if the dataframe is not empty
if orders_df.empty == False:
    fb_db = "orders.json"
    r = requests.get(config.databaseURL + fb_db)
    r = r.json()

    if r:
        data = [r[i] for i in r]
        df_fb = pd.DataFrame.from_dict(data, orient="columns")
        df_fb = df_fb.applymap(str)
    else:
        print("***ERROR- Acess Firebase orders")

    orderIDs_ = df_fb.orderID.values

    ref = db.reference("orders")

    # Compare tuples from Firebase database and tuples from local df.  Only add tuples that do not already exist in Firebase database
    # New record count
    count = 0

    for index, item in orders_df.iterrows():
        if item.orderID not in orderIDs_:
            # push
            ref.push(item.to_dict())

            # Add to new record count
            count += 1

            items = item.values.tolist()

            new_val = ", ".join(repr(e) for e in items)

            new_val2 = new_val.replace("'", "")

            txt_file = open(
                "/Users/jessicaetchechury/Desktop/JEtchCreationsDashboard/Data/orders.txt",
                "a",
            )
            txt_file.write(f"\n{new_val2}")
            txt_file.close()

        else:
            pass
    print(f"Firebase- {count} records added to orders")


else:
    print("DF EMPTY- orders_df")

Firebase- 0 records added to orders


In [13]:
buyers_df = sold_orders_df2[["orderID", "buyerID", "firstName", "lastName"]]


buyer_info_df = pd.merge(buyers_df, buyer_contact, on="orderID", how="outer")
buyer_info_df = buyer_info_df.drop(["Name"], axis=1)
buyer_info_df


buyer_info_df["record_date"] = yesterday_full_date

buyer_info_df = buyer_info_df.applymap(str)
buyer_info_df.fillna("", inplace=True)


print("DF Complete- buyer_info_df")

# Check that records for the day have not already been added
# Do the following if the dataframe is not empty
if buyer_info_df.empty == False:
    fb_db = "buyers.json"
    r = requests.get(config.databaseURL + fb_db)
    r = r.json()

    if r:
        data = [r[i] for i in r]
        df_fb = pd.DataFrame.from_dict(data, orient="columns")
        df_fb = df_fb.applymap(str)
    else:
       
        print("***ERROR- Acess Firebase buyer_info_df")

    df_tups = [(item.buyerID, item.orderID) for index, item in df_fb.iterrows()]

    ref = db.reference("buyers")
    # Compare tuples from Firebase database and tuples from local df.  Only add tuples that do not already exist in Firebase database

    # New record count
    count = 0

    for index, item in buyer_info_df.iterrows():

        # Create tuples from local df
        var_o = item.buyerID, item.orderID

        if var_o not in df_tups:
            # Push item to db
            ref.push(item.to_dict())

            # Add to new record count
            count += 1

            items = item.values.tolist()

            new_val = ", ".join(repr(e) for e in items)

            new_val2 = new_val.replace("'", "")

            txt_file = open(
                "/Users/jessicaetchechury/Desktop/JEtchCreationsDashboard/Data/buyers.txt",
                "a",
            )
            txt_file.write(f"\n{new_val2}")

        else:
            pass

   
    print(f"Firebase- {count} records added to buyers")


else:
   
    print("DF EMPTY- buyer_info_df")

DF Complete- buyer_info_df
Firebase- 0 records added to buyers


In [14]:
os.remove(sold_orders_file_path)

In [15]:
driver.quit()